<a href="https://colab.research.google.com/github/Atsumu-Hiranishi/Chapter2/blob/main/SeeAToDoList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SeeAToDoList.kt

1. SeeATodoListAT クラスと List owners can see their lists テスト


```
@Test
fun `List owners can see their lists`() {

    val listName = "shopping"
    val foodToBuy = listOf("carrots", "apples", "milk")
    val userName = "frank"

    startTheApplication(userName, listName, foodToBuy)

    val list = getToDoList(userName, listName)

    expectThat(list.listName.name).isEqualTo(listName)
    expectThat(list.items.map { it.description }).isEqualTo(foodToBuy)
}
```


- テスト内容: ユーザーが自分のリスト（"shopping"）とそのアイテム（"carrots", "apples", "milk"）を正しく取得できるかを検証する。
- startTheApplication: テストのためにアプリケーションを起動する。
- getToDoList: 指定したユーザーとリスト名に基づいてToDoリストを取得し、期待する内容と一致しているかを検証する。
- expectThat: striktライブラリを使用したアサーションで、リスト名とアイテム内容が期待通りかを確認する。

2. getToDoList メソッド


```
private fun getToDoList(user: String, listName: String): ToDoList {
    val client = JettyClient()
    val response = client(Request(Method.GET, "http://localhost:8081/todo/$user/$listName"))

    return if (response.status == Status.OK)
        parseResponse(response.bodyString())
    else
        fail(response.toMessage())
}
```


- JettyClient: HTTPリクエストを送信するために使用されるクライアント。
- リクエストURL: 指定したユーザーとリスト名でToDoリストを取得するエンドポイントにアクセスする。
- レスポンス確認: ステータスコードが`200 OK`であればレスポンス本文を解析し、ToDoListを返す。それ以外の場合はテストを失敗させる。

3. parseResponse メソッド


```
private fun parseResponse(html: String): ToDoList {
    val nameRegex = "<h2>.*<".toRegex()
    val listName = ListName(extractListName(nameRegex, html))
    val itemsRegex = "<td>.*?<".toRegex()
    val items = itemsRegex.findAll(html)
        .map { ToDoItem(extractItemDesc(it)) }.toList()
    return ToDoList(listName, items)
}
```


- HTML解析: レスポンスのHTML本文からリスト名とアイテムを抽出する。
- nameRegex: リスト名（\<h2>...\</h2>）を抽出する正規表現。
- itemsRegex: 各ToDoアイテム（\<td>...\</td>）を抽出する正規表現。

4. extractListName メソッド


```
private fun extractListName(nameRegex: Regex, html: String): String =
    nameRegex.find(html)?.value
        ?.substringAfter("<h2>")
        ?.dropLast(1)
        .orEmpty()
```


- HTMLからリスト名を抽出するヘルパーメソッド。\<h2>リスト名\</h2>の形式でリスト名を取り出す。

5. extractItemDesc メソッド


```
private fun extractItemDesc(matchResult: MatchResult): String =
    matchResult.value.substringAfter("<td>").dropLast(1)
```


- HTMLの\<td>アイテム\</td>からアイテムの説明を抽出するヘルパーメソッド。

6. startTheApplication メソッド


```
private fun startTheApplication(
    user: String,
    listName: String,
    items: List<String>
) {
    val toDoList = ToDoList(
        ListName(listName),
        items.map(::ToDoItem)
    )
    val lists = mapOf(User(user) to listOf(toDoList))
    val server = Zettai(lists).asServer(Jetty(8081))
    server.start()
}
```


- テスト用サーバーの起動: 指定されたユーザー、リスト名、アイテムでZettaiアプリケーションを起動し、ローカルでJettyサーバーを実行する。
- ポート番号: 8081ポートで起動しており、実際のアプリケーションのメイン設定と分けてテスト環境を設定している。